In [3]:
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml import PipelineModel

In [4]:
# input and output folders
trainingData = "ex_dataregression/trainingData.csv"
unlabeledData = "ex_dataregression/unlabeledData.csv"
outputPath = "predictionsLinearRegressionPipeline/"

In [5]:
# *************************
# Training step
# *************************

# Create a DataFrame from trainingData.csv
# Training data in raw format
trainingData = spark.read.load(trainingData,\
                     format="csv", header=True,\
                     inferSchema=True)

In [6]:
#trainingData.show()

In [7]:
# Define an assembler to create a column (features) of type Vector 
# containing the double values associated with columns attr1, attr2, attr3
assembler = VectorAssembler(inputCols=["attr1", "attr2", "attr3"],\
                            outputCol="features")

In [9]:
# Create a LinearRegression object.  
# LinearRegression is an Estimator that is used to 
# create a regression model based on linear regression
lr = LinearRegression()

In [10]:
# We can set the values of the parameters of the 
# Linear Regression algorithm using the setter methods.
# There is one set method for each parameter
# For example, we are setting the number of maximum iterations to 10
# and the regularization parameter. to 0.0.1
lr.setMaxIter(10)
lr.setRegParam(0.01)

LinearRegression_323af70c14ac

In [11]:
# Define a pipeline that is used to create the linear regression
# model on the training data. The pipeline includes also 
# the preprocessing step
pipeline = Pipeline().setStages([assembler, lr])

In [12]:
# Execute the pipeline on the training data to build the 
# regression model
regressionModel = pipeline.fit(trainingData)

In [14]:
# Now, the regression model can be used to predict the target attribute value
# of new unlabeled data

In [15]:
# Create a DataFrame from unlabeledData.csv
# Unlabeled data in raw format
unlabeledData = spark.read.load(unlabeledData,\
                     format="csv", header=True, inferSchema=True)

In [21]:
#unlabeledData.printSchema()
#unlabeledData.show()

In [17]:
# Make predictions on the unlabled data using the transform() method of the 
# trained regression model transform uses only the content of 'features' 
# to perform the predictions. The model is associated with the pipeline and hence
# also the assembler is executed
predictionsDF = regressionModel.transform(unlabeledData)

In [22]:
#predictionsDF.printSchema()
#predictionsDF.show()

In [19]:
# The returned DataFrame has the following schema (attributes)
# - original attributes
# - features: vector (values of the attributes)
# - label: double (value of the class label)
# - prediction: double (the predicted class label)

# Select only the original features (i.e., the value of the original attributes 
# attr1, attr2, attr3) and the predicted value for each record
predictions = predictionsDF.select("attr1", "attr2", "attr3", "prediction")

In [20]:
# Save the result in an HDFS output folder
predictions.write.csv(outputPath, header="true")